# ElastiCube Tutorial

Welcome to the ElastiCube tutorial! This notebook demonstrates how to:
1. Build OLAP cubes from data
2. Query cubes with SQL-like syntax
3. Perform OLAP operations (slice, dice, drill-down, roll-up)
4. Visualize results with matplotlib
5. Work with Pandas and Polars DataFrames

## Setup

First, let's import the necessary libraries and enable inline plotting.

In [ ]:
import os
from elasticube import ElastiCubeBuilder
import matplotlib.pyplot as plt
import pandas as pd

# Enable inline plotting
%matplotlib inline

print("✓ ElastiCube imported successfully!")

## 1. Building Your First Cube

Let's build a cube from sample sales data. We'll define dimensions (categorical fields) and measures (numeric fields to aggregate).

In [ ]:
# Path to sample data
csv_path = "sales_data.csv"

# Create a cube builder
builder = ElastiCubeBuilder("sales_cube")

# Define dimensions (categorical fields for slicing/dicing)
builder.add_dimension("region", "utf8")
builder.add_dimension("product", "utf8")
builder.add_dimension("category", "utf8")
builder.add_dimension("year", "int64")
builder.add_dimension("quarter", "int64")

# Define measures (numeric fields for aggregation)
builder.add_measure("sales", "float64", "sum")
builder.add_measure("quantity", "int64", "sum")

# Load data from CSV
builder.load_csv(csv_path)

# Build the cube
cube = builder.build()

# Display cube info (Jupyter will show a rich HTML representation)
cube

## 2. Basic Queries

Now let's query the cube! ElastiCube provides a fluent API similar to SQL.

In [ ]:
# Select all data (limited to 10 rows)
query = cube.query()
query.select(["region", "product", "sales", "quantity"])
query.limit(10)

df = query.to_pandas()
df

## 3. Aggregation Queries

Let's perform some aggregations to get sales totals by region.

In [ ]:
# Total sales by region
query = cube.query()
query.select([
    "region",
    "SUM(sales) as total_sales",
    "SUM(quantity) as total_quantity",
    "COUNT(*) as transactions"
])
query.group_by(["region"])
query.order_by(["total_sales DESC"])

df_regional = query.to_pandas()
df_regional

## 4. Filtering Data

Use WHERE-style filters to narrow down your analysis.

In [ ]:
# High-value transactions (sales > 900)
query = cube.query()
query.select(["region", "product", "sales", "quantity"])
query.filter("sales > 900")
query.order_by(["sales DESC"])

df_high_value = query.to_pandas()
df_high_value

## 5. OLAP Operations

### Slice: Filter on a single dimension

In [ ]:
# Slice: Look at only the North region
query = cube.query()
query.filter("region = 'North'")
query.select(["product", "category", "SUM(sales) as total"])
query.group_by(["product", "category"])

df_north = query.to_pandas()
df_north

### Dice: Filter on multiple dimensions

In [ ]:
# Dice: North region + Electronics category
query = cube.query()
query.filter("region = 'North' AND category = 'Electronics'")
query.select(["product", "SUM(sales) as total", "AVG(sales) as avg_sale"])
query.group_by(["product"])

df_dice = query.to_pandas()
df_dice

### Drill-down: Increase granularity

In [ ]:
# Start with regional totals
query = cube.query()
query.select(["region", "SUM(sales) as total"])
query.group_by(["region"])
df_region = query.to_pandas()

print("Regional Level:")
print(df_region)

# Drill down to product level within each region
query = cube.query()
query.select(["region", "product", "SUM(sales) as total"])
query.group_by(["region", "product"])
df_drilldown = query.to_pandas()

print("\nProduct Level (Drilled Down):")
print(df_drilldown)

### Roll-up: Decrease granularity

In [ ]:
# Detailed level: region + product + quarter
query = cube.query()
query.select(["region", "product", "quarter", "SUM(sales) as total"])
query.group_by(["region", "product", "quarter"])
df_detailed = query.to_pandas()

print("Detailed Level (Region × Product × Quarter):")
print(f"Rows: {len(df_detailed)}")
print(df_detailed.head())

# Roll up to just region level
query = cube.query()
query.select(["region", "SUM(sales) as total"])
query.group_by(["region"])
df_rollup = query.to_pandas()

print("\nRolled Up Level (Region Only):")
print(f"Rows: {len(df_rollup)}")
print(df_rollup)

## 6. Visualization with plot()

ElastiCube integrates seamlessly with matplotlib for quick visualizations.

In [ ]:
# Bar chart: Sales by region
query = cube.query()
query.select(["region", "SUM(sales) as total_sales"])
query.group_by(["region"])
query.order_by(["total_sales DESC"])

fig, ax = query.plot().bar(
    x="region",
    y="total_sales",
    title="Total Sales by Region",
    ylabel="Sales ($)",
    color="#4CAF50"
)
plt.show()

In [ ]:
# Line chart: Quarterly trends
query = cube.query()
query.select([
    "quarter",
    "SUM(sales) as total_sales",
    "AVG(sales) as avg_sales"
])
query.group_by(["quarter"])
query.order_by(["quarter ASC"])

fig, ax = query.plot().line(
    x="quarter",
    y=["total_sales", "avg_sales"],
    title="Quarterly Sales Trends",
    xlabel="Quarter",
    ylabel="Sales ($)"
)
plt.show()

In [ ]:
# Pie chart: Market share by region
query = cube.query()
query.select(["region", "SUM(sales) as total"])
query.group_by(["region"])

fig, ax = query.plot().pie(
    labels="region",
    values="total",
    title="Market Share by Region"
)
plt.show()

## 7. Advanced Analysis

Let's combine multiple OLAP operations for deeper insights.

In [ ]:
# Multi-dimensional analysis: Region × Category × Quarter
query = cube.query()
query.select([
    "region",
    "category",
    "quarter",
    "SUM(sales) as total_sales",
    "AVG(sales) as avg_sales",
    "COUNT(*) as transactions"
])
query.group_by(["region", "category", "quarter"])
query.order_by(["total_sales DESC"])
query.limit(15)

df_multi = query.to_pandas()
df_multi

## 8. Working with Polars (High Performance)

For maximum performance, use Polars instead of Pandas!

In [ ]:
try:
    import polars as pl
    
    # Query with Polars (zero-copy from Arrow!)
    query = cube.query()
    query.select(["region", "product", "sales", "quantity"])
    
    df_polars = query.to_polars()
    
    # Use Polars lazy API for additional operations
    result = (
        df_polars.lazy()
        .filter(pl.col("sales") > 900)
        .group_by("region")
        .agg([
            pl.col("sales").sum().alias("total_sales"),
            pl.col("quantity").sum().alias("total_qty"),
            pl.col("sales").mean().alias("avg_sales")
        ])
        .sort("total_sales", descending=True)
        .collect()
    )
    
    print(result)
    
except ImportError:
    print("Polars not installed. Install with: pip install polars")

## 9. Summary

You've learned how to:

✓ Build OLAP cubes from CSV data

✓ Query cubes with SELECT, WHERE, GROUP BY, ORDER BY

✓ Perform OLAP operations (slice, dice, drill-down, roll-up)

✓ Create visualizations with .plot()

✓ Work with both Pandas and Polars DataFrames

✓ Combine multiple dimensions for multi-dimensional analysis

### Next Steps:

- Try loading your own data (CSV, Parquet, or JSON)
- Experiment with different aggregation functions (MIN, MAX, AVG, COUNT DISTINCT)
- Create custom visualizations with matplotlib/seaborn
- Build hierarchies for time-based drill-down (year → quarter → month)

Happy analyzing! 📊